In [61]:
import polars as pl
from benchmark_utils import mock_snakemake

if "snakemake" not in globals() or hasattr(snakemake, "mock"):  # noqa: F821
    snakemake = mock_snakemake("process_generator_data")

In [62]:
# file has whitespaces that we must strip
df0 = pl.read_csv(snakemake.input[0], encoding="iso-8859-1")
df0 = df0.with_row_index(
    name="gen_id"
)  # Order matters, lets add this before anything else happens
df0.columns = [c.strip() for c in df0.columns]
df0 = df0.with_columns(
    pl.col(
        c for c, t in zip(df0.columns, df0.dtypes) if (t == pl.String)
    ).str.strip_chars()
)
# now that they're removed we can convert to numbers
df0 = df0.with_columns(
    pl.col("PlantCode").cast(pl.Int64),
    pl.col("Lat").cast(pl.Float64),
    pl.col("Lon").cast(pl.Float64),
    pl.col("bus").cast(pl.UInt32),
)
df0

gen_id,PlantCode,GenID,bus,FuelType,Pg,Pmax,Pmin,Qg,Qmax,Qmin,Lat,Lon
u32,i64,str,u32,str,f64,f64,i64,i64,f64,f64,f64,f64
0,34,"""1P""",745,"""Conventional Hydroelectric""",4.0333628,12.1,0,0,18.777143,-14.670714,39.134259,-120.953341
1,151,"""1""",1804,"""Petroleum Liquids""",0.0,71.2,0,0,34.5,-23.7,37.62936,-120.93139
2,151,"""2""",1804,"""Natural Gas Fired Combustion T…",0.0,71.2,0,0,34.5,-23.7,37.62936,-120.93139
3,161,"""1""",1964,"""Conventional Hydroelectric""",0.366669,1.1,0,0,18.777143,-14.670714,37.611461,-120.594678
4,161,"""2""",1964,"""Conventional Hydroelectric""",0.366669,1.1,0,0,18.777143,-14.670714,37.611461,-120.594678
…,…,…,…,…,…,…,…,…,…,…,…,…
3887,null,null,8862,null,0.0,0.0,0,0,200.0,-200.0,null,null
3888,null,null,8863,null,0.0,0.0,0,0,200.0,-200.0,null,null
3889,null,null,8864,null,0.0,0.0,0,0,200.0,-200.0,null,null


In [63]:
# Now let's merge with the matpower data to get costs
matpower_gen = pl.read_parquet(
    snakemake.input[1], columns=["gen_id", "bus", "cost_a", "cost_b", "cost_c"]
)
assert matpower_gen.height == df0.height, "Matpower and GIS data doesn't match"
df1 = df0.join(
    matpower_gen, on=["gen_id", "bus"], how="full", coalesce=True, validate="1:1"
)
df1

gen_id,PlantCode,GenID,bus,FuelType,Pg,Pmax,Pmin,Qg,Qmax,Qmin,Lat,Lon,cost_a,cost_b,cost_c
u32,i64,str,i64,str,f64,f64,i64,i64,f64,f64,f64,f64,f32,f32,f32
0,34,"""1P""",745,"""Conventional Hydroelectric""",4.0333628,12.1,0,0,18.777143,-14.670714,39.134259,-120.953341,0.059172,14.878815,46.499573
1,151,"""1""",1804,"""Petroleum Liquids""",0.0,71.2,0,0,34.5,-23.7,37.62936,-120.93139,0.082992,44.91518,1427.413086
2,151,"""2""",1804,"""Natural Gas Fired Combustion T…",0.0,71.2,0,0,34.5,-23.7,37.62936,-120.93139,0.082992,44.91518,1427.413086
3,161,"""1""",1964,"""Conventional Hydroelectric""",0.366669,1.1,0,0,18.777143,-14.670714,37.611461,-120.594678,0.078724,14.878815,34.646786
4,161,"""2""",1964,"""Conventional Hydroelectric""",0.366669,1.1,0,0,18.777143,-14.670714,37.611461,-120.594678,0.078724,14.878815,34.646786
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
3887,null,null,8862,null,0.0,0.0,0,0,200.0,-200.0,null,null,0.0,0.0,0.0
3888,null,null,8863,null,0.0,0.0,0,0,200.0,-200.0,null,null,0.0,0.0,0.0
3889,null,null,8864,null,0.0,0.0,0,0,200.0,-200.0,null,null,0.0,0.0,0.0


In [64]:
# remove reactive elements
assert (df1.get_column("Pmin") == 0).all()
df2 = df1.drop("Pmin")
reactive_elements = df2.filter(pl.col("Pmax") == 0)
assert (
    reactive_elements["FuelType"].is_null().all()
    and (reactive_elements["Qmax"] != 0).all()
), "Expected reactive elements to have Pmax == 0 and Qmax != 0"
df2 = df2.filter(pl.col("Pmax") > 0).drop("Qmax", "Qmin", "Qg")
df2

gen_id,PlantCode,GenID,bus,FuelType,Pg,Pmax,Lat,Lon,cost_a,cost_b,cost_c
u32,i64,str,i64,str,f64,f64,f64,f64,f32,f32,f32
0,34,"""1P""",745,"""Conventional Hydroelectric""",4.0333628,12.1,39.134259,-120.953341,0.059172,14.878815,46.499573
1,151,"""1""",1804,"""Petroleum Liquids""",0.0,71.2,37.62936,-120.93139,0.082992,44.91518,1427.413086
2,151,"""2""",1804,"""Natural Gas Fired Combustion T…",0.0,71.2,37.62936,-120.93139,0.082992,44.91518,1427.413086
3,161,"""1""",1964,"""Conventional Hydroelectric""",0.366669,1.1,37.611461,-120.594678,0.078724,14.878815,34.646786
4,161,"""2""",1964,"""Conventional Hydroelectric""",0.366669,1.1,37.611461,-120.594678,0.078724,14.878815,34.646786
…,…,…,…,…,…,…,…,…,…,…,…
2144,0,"""0""",7583,"""IMPORT""",0.0,200.0,34.288557,-114.103924,0.0,31.395174,0.0
2145,0,"""0""",7604,"""IMPORT""",0.0,200.0,34.086702,-113.914313,0.0,31.395174,0.0
2146,0,"""0""",1293,"""IMPORT""",0.0,200.0,39.52124,-120.008658,0.0,31.395174,0.0


In [65]:
df2.select("FuelType", "Pmax").group_by("FuelType").sum().sort("Pmax", descending=True)

FuelType,Pmax
str,f64
"""Natural Gas Fired Combined Cyc…",19649.3
"""Solar Photovoltaic""",11639.398673
"""Natural Gas Fired Combustion T…",10893.4
"""Conventional Hydroelectric""",7923.0
"""Onshore Wind Turbine""",5303.0
…,…
"""Other Natural Gas""",131.7
"""All Other""",113.7
"""Conventional Steam Coal""",55.0


In [66]:
thermal_fuels = ["coal", "gas", "petroleum", "municipal solid waste"]
df3 = df2.with_columns(
    pl.col("FuelType")
    .str.to_lowercase()
    .map_elements(lambda x: "thermal" if any(f in x for f in thermal_fuels) else x)
    .replace(
        {
            "wood/wood waste biomass": "other renewables",
            "geothermal": "other renewables",
            "other waste biomass": "other renewables",
            "solar photovoltaic": "solar",
            "solar thermal without energy storage": "solar",
            "conventional hydroelectric": "hydro",
            "onshore wind turbine": "wind",
        }
    )
).rename({"FuelType": "type"})
df3.select("type", "Pmax").group_by("type").sum().sort("Pmax", descending=True)

sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.


type,Pmax
str,f64
"""thermal""",36756.9
"""solar""",12795.400002
"""hydro""",7923.0
"""wind""",5303.0
"""import""",5200.0
"""nuclear""",2323.0
"""other renewables""",1768.6
"""hydroelectric pumped storage""",759.1
"""batteries""",229.5


In [67]:
# Remove storage to simplify model and 'all other' since it's negligible
df4 = df3.filter(
    ~pl.col("type").is_in(["batteries", "all other", "hydroelectric pumped storage"])
)
df4.select("type", "Pmax").group_by("type").sum().sort("Pmax", descending=True)

type,Pmax
str,f64
"""thermal""",36756.9
"""solar""",12795.400002
"""hydro""",7923.0
"""wind""",5303.0
"""import""",5200.0
"""nuclear""",2323.0
"""other renewables""",1768.6


In [68]:
df4

gen_id,PlantCode,GenID,bus,type,Pg,Pmax,Lat,Lon,cost_a,cost_b,cost_c
u32,i64,str,i64,str,f64,f64,f64,f64,f32,f32,f32
0,34,"""1P""",745,"""hydro""",4.0333628,12.1,39.134259,-120.953341,0.059172,14.878815,46.499573
1,151,"""1""",1804,"""thermal""",0.0,71.2,37.62936,-120.93139,0.082992,44.91518,1427.413086
2,151,"""2""",1804,"""thermal""",0.0,71.2,37.62936,-120.93139,0.082992,44.91518,1427.413086
3,161,"""1""",1964,"""hydro""",0.366669,1.1,37.611461,-120.594678,0.078724,14.878815,34.646786
4,161,"""2""",1964,"""hydro""",0.366669,1.1,37.611461,-120.594678,0.078724,14.878815,34.646786
…,…,…,…,…,…,…,…,…,…,…,…
2144,0,"""0""",7583,"""import""",0.0,200.0,34.288557,-114.103924,0.0,31.395174,0.0
2145,0,"""0""",7604,"""import""",0.0,200.0,34.086702,-113.914313,0.0,31.395174,0.0
2146,0,"""0""",1293,"""import""",0.0,200.0,39.52124,-120.008658,0.0,31.395174,0.0


In [69]:
# group by type and bus
df5 = (
    df4.drop("Lat", "Lon", "PlantCode", "GenID")
    .group_by("type", "bus")
    .sum()
    .sort("bus", "type")
)
df5

type,bus,gen_id,Pg,Pmax,cost_a,cost_b,cost_c
str,i64,u32,f64,f64,f32,f32,f32
"""hydro""",6,790,1.633345,4.9,0.104841,14.878815,26.207485
"""solar""",16,3247,1.362354,1.939383,0.0,0.0,0.0
"""thermal""",16,3513,0.0,2.8,0.061162,49.007454,496.502167
"""solar""",17,2099,0.999059,1.422214,0.0,0.0,0.0
"""thermal""",17,1724,0.0,1.1,0.030581,24.503727,248.251083
…,…,…,…,…,…,…,…
"""hydro""",8832,670,5.233371,15.7,0.245892,44.636444,100.639664
"""thermal""",8832,7655,0.0,23.0,1.26625,294.535004,867.649963
"""thermal""",8838,855,49.900364,49.9,0.055538,14.512151,283.415802


In [70]:
# write
df5.write_parquet(snakemake.output[0])